Import source

In [2]:
import sys
sys.path.append("src")

Initialize timer

In [4]:
import time
from time import sleep
start_time = time.time()

# 1. Setup Driver

In [6]:
from driver import setup_driver
driver = setup_driver()

# 2. Login KINEO

In [8]:
from login import login
from config import USER_KEY, PASS_KEY
url = 'https://aforadores.mitma.es/contadorestraficofomento/Login.aspx'
username = USER_KEY
password = PASS_KEY
login(driver, url, username, password)

# 3. go to "Informe por horas" 

In [10]:
driver.get('https://aforadores.mitma.es/contadorestraficofomento/InformePorHorasCalzadaCarrilAforo.aspx')

# 4. 'Demarcación' dropdown

In [12]:
from dropdown import select_dropdown_value, get_dropdown_values
input_id_dem = "ctl00_ContentPlaceHolderDatos_CbDemarcacion_I"
dropdown_button_id_dem = "ctl00_ContentPlaceHolderDatos_CbDemarcacion_B-1"
value_to_select_dem = "AP7 BARCELONA SECTOR 4" 
select_dropdown_value(driver, input_id_dem, dropdown_button_id_dem, value_to_select_dem)

sleep(1)

Valor seleccionat: AP7 BARCELONA SECTOR 4



# 5. Set ETD values

In [14]:
dropdown_values_etd = ['AP7-125+650-100270000000', 'AP7-129+500-100280000000', 'AP7-134+050-100290000000']

# 6. Iteration over ETD values, date selector and download Excel result

In [16]:
from button import click_button, download_excel
from dates import select_date
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import pyautogui
from utils import print_elapsed_time, check_no_data_message

# initialize flag
first_iteration = True

for value_to_select_etd in dropdown_values_etd:

    # Select ETD value
    input_id_etd = 'ctl00_ContentPlaceHolderDatos_CbEtd_I'
    dropdown_button_id_etd = 'ctl00_ContentPlaceHolderDatos_CbEtd_B-1'
    select_dropdown_value(driver, input_id_etd, dropdown_button_id_etd, value_to_select_etd)

    sleep(1)
    
    # Initial/End date selector
    input_id_ini = 'LbFechaInicio_I'
    value_to_select_ini = '27/06/2024 00:00:00'
    select_date(driver, input_id_ini, value_to_select_ini)
    
    input_id_fin = 'LbFechaFin_I'
    value_to_select_fin = '28/06/2024 00:00:00'
    select_date(driver, input_id_fin, value_to_select_fin)
    
    sleep(1)

    # Select Desglose value
    input_id_des = "ctl00_ContentPlaceHolderDatos_CbDesglose_I"
    dropdown_button_id_des = "ctl00_ContentPlaceHolderDatos_CbDesglose_B-1"
    value_to_select_des = "CARRIL" 
    select_dropdown_value(driver, input_id_des, dropdown_button_id_des, value_to_select_des)

    sleep(1)

    # Click 'Ver' button
    button_id_ver = "ctl00_ContentPlaceHolderDatos_BtVerListado_I"
    click_button(driver, button_id_ver)

    # Wait for "LoadingPanel" to appear
    sleep(2)

    # Wait until results table is loaded
    while True:
        try:
            # Check if "LoadingPanel" is visible
            WebDriverWait(driver, 1).until( 
                EC.visibility_of_element_located((By.ID, "LoadingPanel"))
            )
        except TimeoutException:
            break

    sleep(2)

    # While message "No hay datos para mostrar" is not shown, Excel will be downloaded
    if not check_no_data_message(driver):

        if first_iteration:
            # 80% zoom adjustment
            pyautogui.keyDown('ctrl')
            pyautogui.press('-')
            pyautogui.press('-')
            pyautogui.keyUp('ctrl')

            first_iteration = False
        
        # Click to download Excel
        button_id_exc = "ctl00_ContentPlaceHolderDatos_BtExcel_I"
        download_excel(driver, button_id_exc) 
        
        sleep(15)  # Wait till download is completed 
        
    else:
        print(f"La ETD {value_to_select_etd} no conté dades a la seva taula. No es baixarà cap document Excel.")

    # printar elapsed time
    print_elapsed_time(start_time)
    print()

driver.quit()

Valor seleccionat: AP7-125+650-100270000000

Valor seleccionat: CARRIL

Temps transcorregut: 0:01:01.946970

Valor seleccionat: AP7-129+500-100280000000

Valor seleccionat: CARRIL

Temps transcorregut: 0:01:30.814741

Valor seleccionat: AP7-134+050-100290000000

Valor seleccionat: CARRIL

Temps transcorregut: 0:01:59.285521

